### 🧠 Orquestación de Agentes IA + Oracle (LangGraph + AISuite)

Este notebook refactoriza la práctica básica a una arquitectura de **Orquestación Multi-Agente** utilizando **LangGraph** para el control de flujo y **AISuite** para la flexibilidad del modelo. El sistema incluye bucles de corrección y manejo de estado compartido (Blackboard).

![Diagrama de Orquestación LangGraph](images/diagram_orquestac_langgraph.png)

#### 🔄 Flujo Orquestado (Máquina de Estado Finito - FSM)
El flujo ya no es lineal, sino un **Grafo Dirigido Acíclico (DAG)** con retroalimentación:
1. **Contextualización (Decisor):** Evalúa la ambigüedad y el esquema. Decide `avanzar`, `repetir` o `guiar`.
2. **Generación SQL:** Crea el código SQL.
3. **Ejecución DB:** Ejecuta y valida el resultado. Decide `correcto`, `error` (a SQL) o `sin_resultados` (a Contextualización).
4. **Interpretación:** Convierte el resultado tabular en lenguaje natural.

In [ ]:
# 🔐 CONFIGURACIÓN E INICIALIZACIÓN
import os
import json
import oracledb
import pandas as pd
from dotenv import load_dotenv
from typing import TypedDict, Callable, Literal, TypeAlias, cast
from langgraph.graph import StateGraph, END
import aisuite as ai

# Cargar variables de entorno
load_dotenv()
print("Variables de entorno cargadas.")

# Inicializar cliente AISuite con type hint explícito
client: ai.Client = ai.Client()

# --- Variables de Modelo (Aprovechando la flexibilidad de AISuite) ---
MODEL_CONTEXTUALIZACION: str = "google:gemini-2.0-flash-001"  # Ideal para decisiones rápidas y salida estructurada
MODEL_GENERACION_SQL: str = "openai:gpt-4o"  # Ideal para tareas de codificación precisas
MODEL_INTERPRETACION: str = "openai:gpt-3.5-turbo"  # Ideal para síntesis y explicación

# --- Tipos de control de flujo (sintaxis moderna 3.10+) ---
Transition: TypeAlias = Literal["iniciar", "avanzar", "repetir", "finalizado"]
DBSignal: TypeAlias = Literal["correcto", "error", "sin_resultados"]

# Globales de Conexión (con type hints modernos Python 3.10+)
ORACLE_USER: str | None = os.getenv("ORACLE_USER")
ORACLE_HOST: str | None = os.getenv("ORACLE_HOST")
ORACLE_PORT: str | None = os.getenv("ORACLE_PORT")
ORACLE_SID: str | None = os.getenv("ORACLE_SID")
ORACLE_PASSWORD: str | None = os.getenv("ORACLE_PASSWORD")

# Globales de Estado y Conexión
DB_SCHEMA: str = ""
ORACLE_CONNECTION: oracledb.Connection | None = None


def _require(name: str, value: str | None) -> str:
    """Devuelve el valor si no es None/ vacío; lanza ValueError en caso contrario.
    Esto permite estrechar los tipos (de str | None a str) de forma segura.
    """
    if value is None or value.strip() == "":
        raise ValueError(f"Variable de entorno requerida no definida: {name}")
    return value


def get_oracle_connection_and_schema() -> tuple[oracledb.Connection, oracledb.Cursor]:
    """Establece la conexión a Oracle y obtiene el esquema de la BD.
    
    Esta función es llamada una vez al inicio del grafo.
    
    Returns:
        Tupla con la conexión y el cursor de Oracle
        
    Note:
        Esta función usa globales por necesidad de inicialización única,
        pero los agentes NO usarán estas globales directamente.
    """
    global DB_SCHEMA, ORACLE_CONNECTION
    
    if ORACLE_CONNECTION is None:
        # Validar y estrechar tipos de entorno a str
        host: str = _require("ORACLE_HOST", ORACLE_HOST)
        port_str: str = _require("ORACLE_PORT", ORACLE_PORT)
        sid: str = _require("ORACLE_SID", ORACLE_SID)
        user: str = _require("ORACLE_USER", ORACLE_USER)
        password: str = _require("ORACLE_PASSWORD", ORACLE_PASSWORD)

        # Convertir puerto a entero (makedsn acepta int)
        try:
            port: int = int(port_str)
        except ValueError as e:
            raise ValueError("ORACLE_PORT debe ser un entero válido") from e

        # Construir DSN con tipos correctos (sin None)
        dsn: str = oracledb.makedsn(host, port, sid=sid)
        ORACLE_CONNECTION = oracledb.connect(user=user, password=password, dsn=dsn)
    
    cursor: oracledb.Cursor = ORACLE_CONNECTION.cursor()
    
    # Obtener esquema de la base de datos (Adaptado de su código)
    cursor.execute(f"""
        SELECT table_name, column_name
        FROM all_tab_columns
        WHERE owner = UPPER('{ORACLE_USER}')
        ORDER BY table_name, column_id
    """)
    
    esquema_dict: dict[str, list[str]] = {}
    for table, column in cursor:
        esquema_dict.setdefault(table, []).append(column)
        
    # Formato legible para el LLM
    DB_SCHEMA = "\n".join([f"- {table}({', '.join(cols).lower()})" for table, cols in esquema_dict.items()])
    print("\n[INFO] Conexión Oracle y Esquema obtenidos.\n")
    return ORACLE_CONNECTION, cursor

# Llamada inicial para establecer la conexión y esquema
get_oracle_connection_and_schema()


In [ ]:
# Verificación rápida de conexión correcta con  base de datos
print(DB_SCHEMA)

### 💾 Definición del Blackboard (Estado Compartido)
Esta es la estructura de datos (`TypedDict`) que pasa entre todos los agentes, manteniendo el estado de la tarea.

In [ ]:
class AgentState(TypedDict):
    """Define el estado compartido (Blackboard) que se pasa entre los nodos.
    
    Note:
        Usa sintaxis moderna de tipos (Python 3.10+) y alias de tipos para transiciones.
    """
    pregunta_original: str
    pregunta_mejorada: str | None
    sql_query: str | None
    resultado_db_raw: str | None
    respuesta_final: str | None
    transicion: Transition  # Señal de control para LangGraph
    # Conexión y esquema (se pasan como parte del estado para evitar globales)
    db_connection: oracledb.Connection | None
    db_schema: str

### 🤖 Funciones de los Agentes (Nodos del Grafo)

Cada agente está diseñado con **MÁXIMA DISCIPLINA DE TIPADO** y **MEJORES PRÁCTICAS**:

#### 📘 Principios Pedagógicos Implementados:

1. **Type Hints Obligatorios en TODO** (Sintaxis moderna Python 3.10+)
   ```python
   def agente_contextualizacion(state: AgentState) -> AgentState:
       pregunta: str = state["pregunta_original"]
       db_schema: str = state.get("db_schema", "")
       # Usa | None en lugar de Optional[tipo]
       resultado: str | None = None
   ```

2. **UN SOLO return por función** (o máximo 2 muy justificados)
   - ❌ Evitamos múltiples returns que confunden el flujo
   - ✅ Usamos variables de resultado que se devuelven al final

3. **Guard Clauses al inicio** (Validaciones tempranas)
   ```python
   if not sql_code:
       return "error"
   ```

4. **Funciones auxiliares privadas** (prefijo `_`)
   - Separan lógica compleja en piezas testables
   - Ejemplo: `_parse_json_response()`, `_clean_sql_response()`

5. **Documentación completa** (Docstrings con Args, Returns, Raises)

6. **Manejo de recursos con finally**
   ```python
   finally:
       if cursor is not None:
           cursor.close()
   ```

### 📚 Conceptos Clave de LangGraph (Para Estudiantes)

Este notebook se centra en **Orquestación Multi-Agente con LangGraph**. Los principios de buenas prácticas de Python se encuentran en `buenaspracticas.ipynb`.

#### 🔑 Conceptos Fundamentales de LangGraph:

**1️⃣ Nodos (Agents):** Funciones que procesan el estado
```python
def agente_contextualizacion(state: AgentState) -> AgentState:
    # Procesa y actualiza el estado
    state["pregunta_mejorada"] = procesada
    return state  # DEBE devolver AgentState (dict)
```

**2️⃣ Estado Compartido (Blackboard Pattern):**
```python
class AgentState(TypedDict):
    pregunta_original: str
    transicion: Transition  # Señal de control
    db_connection: oracledb.Connection | None
```

**3️⃣ Transiciones Condicionales:**
```python
workflow.add_conditional_edges(
    "contextualizacion",
    lambda x: x["transicion"],  # Función de routing
    {
        "avanzar": "generacion_sql",
        "repetir": "contextualizacion"  # Bucle
    }
)
```

**4️⃣ Funciones de Routing Separadas:**
- Los nodos **SIEMPRE** devuelven `AgentState`
- Las funciones de routing **leen el estado** y devuelven la clave de transición
```python
def _route_ejecucion_db(state: AgentState) -> DBSignal:
    return cast(DBSignal, state.get("transicion", "error"))
```

**5️⃣ Tipos de Control de Flujo:**
```python
Transition: TypeAlias = Literal["iniciar", "avanzar", "repetir", "finalizado"]
DBSignal: TypeAlias = Literal["correcto", "error", "sin_resultados"]
```

---

### 🎯 Patrón de Orquestación Completa

```python
# 1. Definir el grafo
workflow = StateGraph(AgentState)

# 2. Registrar nodos (agentes)
workflow.add_node("agente1", funcion_agente1)
workflow.add_node("agente2", funcion_agente2)

# 3. Definir punto de entrada
workflow.set_entry_point("agente1")

# 4. Transiciones condicionales
workflow.add_conditional_edges(
    "agente1",
    funcion_routing,  # Lee state y devuelve clave
    {"opcion1": "agente2", "opcion2": END}
)

# 5. Compilar y ejecutar
app = workflow.compile()
resultado = app.invoke(estado_inicial)
```

**🎓 Para buenas prácticas generales de Python (type hints, docstrings, etc.), consulta `buenaspracticas.ipynb`**

In [ ]:
# --- NODO 1: Agente de Contextualización ---
def agente_contextualizacion(state: AgentState) -> AgentState:
    """Evalúa la pregunta inicial y decide si puede procesarse o necesita clarificación.
    
    Args:
        state: Estado compartido con la pregunta original y esquema de BD
        
    Returns:
        Estado actualizado con la pregunta mejorada y señal de transición
    """
    pregunta: str = state["pregunta_original"]
    db_schema: str = state.get("db_schema", "")
    
    prompt: str = f"""
    [SISTEMA] Eres el Agente de Contextualización. Tu función es evaluar si la 'consulta_usuario' puede ser respondida con el esquema Oracle. DEBES devolver EXCLUSIVAMENTE un objeto JSON. (Modelo: {MODEL_CONTEXTUALIZACION})

    [REGLAS]
    1. Si la consulta es ambigua o no coincide con el esquema, transición es 'repetir' y 'pregunta_mejorada' debe ser una pregunta de clarificación para el usuario.
    2. Si la consulta es clara, transición es 'avanzar' y 'pregunta_mejorada' es la versión optimizada para el agente SQL.

    [ESQUEMA JSON REQUERIDO]
    {{ "transicion": "avanzar" | "repetir", "pregunta_mejorada": "string" }}

    [INPUTS]
    Consulta del Usuario: {pregunta}
    Esquema de la Base de Datos:
{db_schema}
    """
    
    response = client.chat.completions.create(
        model=MODEL_CONTEXTUALIZACION,
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Parsing del JSON de control con manejo robusto de errores
    llm_output: dict[str, str] = _parse_json_response(response.choices[0].message.content)
    
    # Actualizar estado y retornar (UN SOLO return)
    state.update({
        "pregunta_mejorada": llm_output["pregunta_mejorada"], 
        "transicion": llm_output["transicion"]  # type: ignore[typeddict-item]
    })
    return state


def _parse_json_response(raw_output: str) -> dict[str, str]:
    """Función auxiliar: Parsea la respuesta JSON del LLM con manejo de errores.
    
    Args:
        raw_output: Texto crudo de la respuesta del LLM
        
    Returns:
        Diccionario con las claves 'transicion' y 'pregunta_mejorada'
        
    Note:
        Esta función aísla la lógica de parsing para mejorar la testabilidad
        y claridad del código principal.
    """
    try:
        cleaned_output: str = raw_output.strip()
        
        # Eliminar bloques de código Markdown si existen
        if cleaned_output.startswith("```"):
            cleaned_output = "\n".join(cleaned_output.split("\n")[1:-1]).strip()
        
        parsed_json: dict[str, str] = json.loads(cleaned_output)
        return parsed_json
        
    except (json.JSONDecodeError, KeyError, IndexError) as e:
        print(f"[ERROR] Fallo en parsing JSON: {e}. Forzando 'repetir'.")
        # Fallback seguro en caso de error
        return {
            "transicion": "repetir", 
            "pregunta_mejorada": "Error interno. Por favor, reformule su consulta."
        }


# --- NODO 2: Agente de Generación SQL ---
def agente_generacion_sql(state: AgentState) -> AgentState:
    """Genera una consulta SQL válida basada en la pregunta mejorada.
    
    Args:
        state: Estado compartido con la pregunta mejorada y esquema
        
    Returns:
        Estado actualizado con la query SQL generada
    """
    pregunta_limpia: str | None = state["pregunta_mejorada"]
    db_schema: str = state.get("db_schema", "")
    
    messages: list[dict[str, str]] = [
        {
            "role": "system", 
            "content": f"Eres un asistente experto en SQL para Oracle. Genera solo la consulta SQL compatible con Oracle, sin NINGÚN texto explicativo o formato Markdown. Usa este esquema:\n{db_schema}"
        },
        {"role": "user", "content": str(pregunta_limpia)}
    ]
    
    response = client.chat.completions.create(
        model=MODEL_GENERACION_SQL,
        messages=messages
    )
    
    # Limpiar y normalizar el SQL generado
    sql_generado: str = _clean_sql_response(response.choices[0].message.content)
    
    state["sql_query"] = sql_generado
    return state


def _clean_sql_response(raw_sql: str) -> str:
    """Función auxiliar: Limpia la respuesta SQL del LLM.
    
    Args:
        raw_sql: SQL crudo que puede contener formato Markdown
        
    Returns:
        SQL limpio y normalizado en una sola línea
    """
    cleaned_sql: str = raw_sql.strip()
    
    # Eliminar bloques de código Markdown
    if cleaned_sql.startswith("```"):
        cleaned_sql = "\n".join(cleaned_sql.split("\n")[1:-1]).strip()
    
    # Normalizar: eliminar punto y coma final y reducir espacios
    cleaned_sql = cleaned_sql.replace(";", "").replace("\n", " ").replace("\t", " ")
    
    # Eliminar espacios múltiples
    return " ".join(cleaned_sql.split())


# --- NODO 3: Agente de Ejecución DB ---
def agente_ejecucion_db(state: AgentState) -> AgentState:
    """Ejecuta la query SQL y actualiza el estado con resultados.
    
    Args:
        state: Estado compartido con la query SQL y conexión a BD
        
    Returns:
        Estado actualizado con resultados y señal de control
        
    Note:
        LangGraph requiere que TODOS los nodos devuelvan AgentState (dict).
        La señal de control se almacena en state["transicion"] para el router.
    """
    sql_code: str | None = state.get("sql_query", "")
    db_connection: oracledb.Connection | None = state.get("db_connection")
    
    # Validaciones tempranas (Guard Clauses)
    if not sql_code:
        print("[ERROR] No hay SQL para ejecutar. Señal: error")
        state["transicion"] = "error"  # type: ignore[typeddict-item]
        return state
    
    if db_connection is None:
        print("[ERROR] Conexión a base de datos no disponible. Señal: error")
        state["transicion"] = "error"  # type: ignore[typeddict-item]
        return state
    
    # Ejecutar query y actualizar estado con señal de control
    resultado_señal: DBSignal = _execute_query_and_update_state(sql_code, db_connection, state)
    state["transicion"] = resultado_señal  # type: ignore[typeddict-item]
    return state


def _execute_query_and_update_state(
    sql_code: str, 
    db_connection: oracledb.Connection, 
    state: AgentState
) -> DBSignal:
    """Función auxiliar: Ejecuta SQL y actualiza el estado con resultados.
    
    Args:
        sql_code: Consulta SQL a ejecutar
        db_connection: Conexión activa a Oracle
        state: Estado compartido a actualizar
        
    Returns:
        Señal de control según el resultado de la ejecución
    """
    cursor: oracledb.Cursor | None = None
    resultado_señal: DBSignal = "error"  # Inicializar con valor por defecto
    
    try:
        cursor = db_connection.cursor()
        cursor.execute(sql_code)
        resultados: list = cursor.fetchall()
        
        # Procesar resultados en DataFrame
        columnas: list[str] = [col[0] for col in cursor.description]  # type: ignore
        df: pd.DataFrame = pd.DataFrame(resultados, columns=columnas)
        
        # Convertir a Markdown y actualizar estado
        texto_resultado: str = df.to_markdown(index=False)
        state["resultado_db_raw"] = texto_resultado
        
        # Determinar señal según contenido
        if df.empty:
            print("[INFO] Ejecución: 0 filas. Señal: sin_resultados")
            resultado_señal = "sin_resultados"
        else:
            print("[INFO] Ejecución: Éxito con datos. Señal: correcto")
            resultado_señal = "correcto"
            
    except oracledb.DatabaseError as e:
        print(f"[ERROR] Error de base de datos Oracle: {e}. Señal: error")
        resultado_señal = "error"
        
    except Exception as e:
        print(f"[ERROR] Error de ejecución genérico: {e}. Señal: error")
        resultado_señal = "error"
        
    finally:
        # Garantizar cierre de recursos
        if cursor is not None:
            cursor.close()
    
    return resultado_señal


def _route_ejecucion_db(state: AgentState) -> DBSignal:
    """Función de routing: Lee la señal de control del estado.
    
    Args:
        state: Estado con la señal de transición almacenada
        
    Returns:
        Señal de control para LangGraph: 'correcto', 'error' o 'sin_resultados'
        
    Note:
        LangGraph requiere funciones de routing separadas que lean el estado
        y devuelvan la clave de la siguiente transición.
    """
    # Casting seguro porque sabemos que transicion contiene un DBSignal en este punto
    señal: str = state.get("transicion", "error")  # type: ignore[assignment]
    return cast(DBSignal, señal)


# --- NODO 4: Agente de Interpretación ---
def agente_interpretacion(state: AgentState) -> AgentState:
    """Interpreta los resultados de la BD en lenguaje natural.
    
    Args:
        state: Estado con la pregunta original y resultados de la consulta
        
    Returns:
        Estado actualizado con la respuesta final en lenguaje natural
    """
    pregunta: str = state["pregunta_original"]
    resultado_raw: str | None = state["resultado_db_raw"]
    db_schema: str = state.get("db_schema", "")
    
    messages: list[dict[str, str]] = [
        {
            "role": "system", 
            "content": f"Eres un experto en análisis de datos. Resume e interpreta los resultados de una consulta SQL realizada sobre este esquema: {db_schema}"
        },
        {
            "role": "user", 
            "content": f"La consulta original fue: {pregunta}\n\nY los resultados en formato Markdown fueron:\n\n{resultado_raw}"
        }
    ]
    
    response = client.chat.completions.create(
        model=MODEL_INTERPRETACION,
        messages=messages
    )
    
    respuesta_final: str = response.choices[0].message.content.strip()
    
    # Actualizar estado con respuesta final y señal de finalización
    state.update({
        "respuesta_final": respuesta_final,
        "transicion": "finalizado"
    })
    return state
print("Definiciones de agentes realizadas.")

### 🧠 LangGraph: El Agente Orquestador
Aquí se define el flujo de trabajo (el grafo), incluyendo los bucles de corrección que hacen robusto el sistema.

In [ ]:
def setup_and_run_app(initial_input: dict) -> AgentState:
    """Define y ejecuta la aplicación LangGraph con validación estricta.
    
    Args:
        initial_input: Diccionario con configuración inicial que DEBE contener:
                      - 'pregunta_original': str
                      - 'db_connection': oracledb.Connection
                      - 'db_schema': str
    
    Returns:
        Estado final después de ejecutar el grafo completo
        
    Raises:
        ValueError: Si faltan claves requeridas o valores son inválidos
    """
    # Validaciones tempranas (Guard Clauses) - CRÍTICO para enseñar a los alumnos
    _validate_initial_input(initial_input)
    
    # Construir y configurar el grafo de orquestación
    workflow: StateGraph = StateGraph(AgentState)
    
    # Paso 1: Registrar los nodos (agentes)
    workflow.add_node("contextualizacion", agente_contextualizacion)
    workflow.add_node("generacion_sql", agente_generacion_sql)
    workflow.add_node("ejecucion_db", agente_ejecucion_db)
    workflow.add_node("interpretacion", agente_interpretacion)
    
    # Paso 2: Definir punto de entrada
    workflow.set_entry_point("contextualizacion")
    
    # Paso 3: Definir transiciones condicionales
    _add_workflow_edges(workflow)
    
    # Paso 4: Compilar y ejecutar
    app = workflow.compile()
    
    print(f"[INFO] Iniciando LangGraph para: {initial_input['pregunta_original']}")
    final_state: AgentState = app.invoke(initial_input)
    
    return final_state


def _validate_initial_input(initial_input: dict) -> None:
    """Función auxiliar: Valida la estructura del input inicial.
    
    Args:
        initial_input: Diccionario a validar
        
    Raises:
        ValueError: Si falta alguna clave requerida o los valores son inválidos
    """
    if "pregunta_original" not in initial_input:
        raise ValueError("El input debe contener 'pregunta_original'")
    
    if initial_input.get("db_connection") is None:
        raise ValueError("El input debe contener una conexión válida 'db_connection'")
    
    if not initial_input.get("db_schema"):
        raise ValueError("El input debe contener 'db_schema' no vacío")


def _add_workflow_edges(workflow: StateGraph) -> None:
    """Función auxiliar: Configura las transiciones del grafo.
    
    Args:
        workflow: Grafo de estados a configurar
        
    Note:
        Esta función aísla la lógica de configuración del grafo para mejorar
        la legibilidad del código principal.
    """
    # Transiciones desde Contextualización
    workflow.add_conditional_edges(
        "contextualizacion", 
        lambda x: x["transicion"], 
        {
            "avanzar": "generacion_sql",
            "repetir": "contextualizacion",  # Bucle de clarificación
        }
    )
    
    # Transición desde Generación SQL
    workflow.add_edge("generacion_sql", "ejecucion_db")
    
    # Transiciones desde Ejecución DB (USA FUNCIÓN DE ROUTING SEPARADA)
    workflow.add_conditional_edges(
        "ejecucion_db", 
        _route_ejecucion_db,  # Función que lee state["transicion"]
        {
            "correcto": "interpretacion",
            "error": "generacion_sql",          # Bucle de corrección SQL
            "sin_resultados": "contextualizacion",  # Bucle de realimentación
        }
    )
    
    # Transición de salida
    workflow.add_conditional_edges(
        "interpretacion", 
        lambda x: x["transicion"], 
        {"finalizado": END}
    )
print("Transiciones del grafo configuradas.")

### 🚀 EJECUCIÓN Y PRUEBA
Ejecute esta celda para probar el flujo orquestado.

In [ ]:
# 🧠 Entrada en lenguaje natural.
pregunta_usuario = "Quiero saber los productos que ha comprado Ana Gamez y lo que le costó cada uno"

# Inicializar estado con toda la información necesaria (Mejor práctica)
initial_input = {
    "pregunta_original": pregunta_usuario,
    "transicion": "iniciar",
    "db_connection": ORACLE_CONNECTION,  # Conexión desde la celda de inicialización
    "db_schema": DB_SCHEMA,              # Esquema desde la celda de inicialización
    "pregunta_mejorada": None,
    "sql_query": None,
    "resultado_db_raw": None,
    "respuesta_final": None
}

final_state = setup_and_run_app(initial_input)

print("\n" + "="*60)
print("✅ RESUMEN DE LA EJECUCIÓN COMPLETA")
print("="*60)
print(f"-> Pregunta Inicial: {final_state.get('pregunta_original')}")
print(f"-> Pregunta Mejorada: {final_state.get('pregunta_mejorada')}")
print(f"-> SQL Generado: {final_state.get('sql_query')}")
print("\n--- RESULTADO DB ---")
print(final_state.get('resultado_db_raw'))
print("\n--- RESPUESTA FINAL ---")
print(final_state.get('respuesta_final'))
print("""
[NOTA EDUCATIVA] 
✅ Este código sigue las mejores prácticas:
1. No usa variables globales dentro de los agentes
2. Valida precondiciones antes de ejecutar
3. Maneja errores de forma explícita
4. Cierra recursos (cursores) en finally
5. El estado (Blackboard) contiene toda la información necesaria

Para probar los bucles de corrección, modifique la lógica de simulación 
en agente_ejecucion_db para forzar 'error' o 'sin_resultados' en diferentes 
pasadas del grafo.
""")

In [ ]:
# ✅ Comprobación de versión mínima de Python
import sys

required = (3, 13, 3)
current = sys.version_info[:3]
assert current >= required, f"Se requiere Python {required[0]}.{required[1]}.{required[2]} o superior. Detectado: {current[0]}.{current[1]}.{current[2]}"
print(f"[OK] Python {current[0]}.{current[1]}.{current[2]} >= {required[0]}.{required[1]}.{required[2]}")
